# Gridpoint env precursor analysis


* So far using CAPE/TCWV
* 1d/2d histograms
* Filter on fractional growth rate values.
* My initial impressions are that neither CAPE nor TCWV are strongly correlated with MCS area, or growth rates.
* TODO: Try subsetting tracks to e.g. tropics?


* 19/8/22: Initial work. Basic match up of tracks data and era5 tracks linked data.

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import xarray as xr

from mcs_prime import McsTracks
from mcs_prime.mcs_prime_config import PATHS

/home/users/mmuetz/miniconda3/envs/mcs_prime_env/lib/python3.10/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [ ]:
tracks = McsTracks.open(PATHS['statsdir'] / 'mcs_tracks_final_extc_20190101.0000_20200101.0000.nc', 
                        PATHS['pixeldir'])
ntracks = len(tracks.dstracks.tracks)

In [ ]:
# Produced by remakefiles/track_era5_env_precursor.py
track_era5 = xr.open_dataset(PATHS['datadir'] / 'mcs_prime_output/track_era5_env_precursor/2019' / 'daily_track_era5_data_2019.nc')

In [ ]:
mask = ~np.isnan(track_era5.cape)
m2 = ~np.isnan(track_era5.tcwv)
(mask == m2).all()

In [ ]:
plt.hist(track_era5.cape.values[mask], bins=np.linspace(0, 4000, 200));

Large spike at CAPE=0

In [ ]:
plt.hist(track_era5.tcwv.values[mask], bins=np.linspace(0, 100, 200));

In [ ]:
plt.hist2d(tracks.dstracks.area.values[mask], track_era5.cape.values[mask], bins=(np.linspace(0, 1e5, 100), np.linspace(0, 2000, 100)));

In [ ]:
plt.hist2d(tracks.dstracks.area.values[mask], track_era5.tcwv.values[mask], bins=(np.linspace(0, 1e5, 100), np.linspace(50, 80, 100)));

In [ ]:
dt = 1
fractional_area_growth = tracks.dstracks.area.values.copy()
fractional_area_growth[:, 1:] = (1 / tracks.dstracks.area[:, 1:].values) * ((tracks.dstracks.area[:, 1:].values - tracks.dstracks.area[:, :-1].values) / dt)

In [ ]:
area_mask = mask.copy()
area_mask[:, 0] = False

In [ ]:
plt.hist2d(fractional_area_growth[area_mask], track_era5.cape.values[area_mask], bins=(np.linspace(-1, 1, 100), np.linspace(0, 1000, 100)));

In [ ]:
plt.hist2d(fractional_area_growth[area_mask], track_era5.tcwv.values[area_mask], bins=(np.linspace(-1, 1, 100), 100), norm=LogNorm());

In [ ]:
plt.hist2d(fractional_area_growth[area_mask], track_era5.cape.values[area_mask], bins=(np.linspace(-1, 1, 100), np.linspace(100, 2000, 100)), norm=LogNorm());

In [ ]:
early_mask = area_mask.copy()
early_mask[:, 4:] = False

In [ ]:
plt.hist2d(fractional_area_growth[early_mask], track_era5.tcwv.values[early_mask], bins=(np.linspace(-1, 1, 100), 100), norm=LogNorm());

In [ ]:
plt.hist2d(fractional_area_growth[early_mask], track_era5.cape.values[early_mask], bins=(np.linspace(-1, 1, 100), np.linspace(100, 2000, 100)), norm=LogNorm());

In [ ]:
plt.hist2d(fractional_area_growth[early_mask], track_era5.cape.values[early_mask], bins=(np.linspace(-1, 1, 100), np.linspace(100, 2000, 100)));

In [ ]:
thresh = 0.1  # roughly equal numbers in each.
growth_mask = fractional_area_growth > thresh
decay_mask = fractional_area_growth < -thresh
stable_mask = (fractional_area_growth >= -thresh) & (fractional_area_growth <= thresh)
print(growth_mask.sum())
print(stable_mask.sum())
print(decay_mask.sum())

In [ ]:
plt.figure(figsize=(10, 8))
for l, m, c in [('growth', growth_mask, 'b'), ('stable', stable_mask, 'r'), ('decay', decay_mask, 'g')]:
    data = track_era5.cape.values[m & mask]
    mean = data[data > 100].mean()
    p = plt.hist(data, bins=np.linspace(100, 3000, 200), histtype='step', density=True, label=l, color=c);
    plt.axvline(x=mean, color=c)
    print(mean)
plt.legend()


In [ ]:
plt.figure(figsize=(10, 8))
for l, m, c in [('growth', growth_mask, 'b'), ('stable', stable_mask, 'r'), ('decay', decay_mask, 'g')]:
    data = track_era5.tcwv.values[m & mask]
    mean = data.mean()
    p = plt.hist(data, bins=np.linspace(0, 80, 200), histtype='step', density=True, label=l, color=c);
    plt.axvline(x=mean, color=c)
    print(mean)
plt.legend()
